In [ ]:
# install yfinance (no output printed)
!pip install yfinance > /dev/null 2>&1

In [1]:
# IMPORTS
import yfinance as yf
import datetime
import os
import pandas as pd

In [2]:
# ---- CONFIGURATION ----
symbols = ["AAPL", "MSFT", "NVDA"]  # <-- Edit this list as needed
base_save_dir = "../data"             # Base folder to save CSVs
years_back = 5                        # Number of years of history to fetch
# ------------------------

In [4]:
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365 * years_back)

os.makedirs(base_save_dir, exist_ok=True)

for symbol in symbols:
    symbol_save_dir = os.path.join(base_save_dir, symbol)
    os.makedirs(symbol_save_dir, exist_ok=True) # Create a subdirectory for each symbol

    print(f"Fetching {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=True)
    
    if not data.empty:
        # Reset index to make Date a regular column
        data = data.reset_index()
        
        # Handle MultiIndex columns properly
        if isinstance(data.columns, pd.MultiIndex):
            # For MultiIndex, extract just the first level (the actual column names)
            data.columns = [col[0] if col[0] else col[1] for col in data.columns]
        
        # Rename 'Adj Close' to 'Price' for consistency (if it exists)
        if 'Adj Close' in data.columns:
            data = data.rename(columns={'Adj Close': 'Price'})
        
        # Reorder columns to match expected format: Date, Price, Close, High, Low, Open, Volume
        # Create the desired column order, but only include columns that actually exist
        desired_order = ['Date', 'Price', 'Close', 'High', 'Low', 'Open', 'Volume']
        available_columns = [col for col in desired_order if col in data.columns]
        data = data[available_columns]
        
        file_path = os.path.join(symbol_save_dir, f"{symbol}_{start_date}_{end_date}.csv")
        data.to_csv(file_path, index=False)
        print(f"✅ Saved {symbol} data to {file_path}")
    else:
        print(f"⚠️ No data returned for {symbol}")

print("Done.")

Fetching AAPL...
✅ Saved AAPL data to ../data/AAPL/AAPL_2020-10-17_2025-10-16.csv
Fetching MSFT...
✅ Saved MSFT data to ../data/MSFT/MSFT_2020-10-17_2025-10-16.csv
Fetching NVDA...
✅ Saved NVDA data to ../data/NVDA/NVDA_2020-10-17_2025-10-16.csv
Done.
